In [1]:
import nest_asyncio
import os

from llama_index.llms.openai import OpenAI
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex

from IPython.display import display, Markdown

from llama_index.core.tools import QueryEngineTool

In [2]:
nest_asyncio.apply()

In [3]:
index = LlamaCloudIndex(
  name=os.getenv("INDEX_NAME"), 
  project_name="Default",
  organization_id=os.getenv("ORG_ID"),
  api_key=os.getenv("LLAMA_API_KEY")
)

In [4]:
query = "What about DC?"

In [5]:
response = index.as_query_engine().query(query)

In [7]:
response.response

'Washington, D.C. is the capital city and federal district of the United States. It was named after George Washington, the first president of the United States, and is not part of any state. The city was founded in 1791 and serves as the seat of the U.S. federal government. Washington, D.C. is known for its historical significance, hosting various national monuments, museums, and government buildings. Additionally, the city is recognized for its diverse culture, arts scene, cuisine, and as a major political and tourist destination.'

In [ ]:
display(Markdown(response.response))

Washington, D.C. is the capital city and federal district of the United States. It was named after George Washington, the first president of the United States, and is not part of any state. The city was founded in 1791 and serves as the seat of the U.S. federal government. Washington, D.C. is known for its historical significance, hosting various federal government headquarters, national monuments, museums, and international organizations. Additionally, the city is recognized for its diverse culture, arts scene, cuisine, and as a major tourist destination.

## Add memory so that you can have a conversation with the bot

In [23]:
from llama_index.core.chat_engine import ContextChatEngine
from llama_index.core.memory import ChatMemoryBuffer

# Get retriever from the index
retriever = index.as_retriever()

# Set up memory
memory = ChatMemoryBuffer.from_defaults(token_limit=2000)

# Create chat engine with memory
chat_engine = ContextChatEngine.from_defaults(
    retriever=retriever,
    memory=memory,
    llm=OpenAI(model="gpt-4o"),
)

# Then to query:
response = chat_engine.chat("What is the population of New York?")
print(response.response)

As of 2024, the estimated population of New York is 19,867,248.


In [24]:
response = chat_engine.chat("What about California?")
print(response.response)

As of 2024, the population of California is 39,431,263.
